In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [10]:
df = pd.read_csv("com.csv")

In [11]:
df

,index,complaint,Product,target
0,0,This auto loan was opened on XX/XX/2020 in XXX...,Vehicle loan or lease,5
1,1,In XXXX of 2019 I noticed a debt for {$620.00}...,Debt collection,0
2,2,"As stated from Capital One, XXXX XX/XX/XXXX an...",Vehicle loan or lease,5
3,3,"Please see CFPB case XXXX. \n\nCapital One, in...",Checking or savings account,3
4,4,This debt was incurred due to medical malpract...,Debt collection,0
...,...,...,...,...
6994,6994,"I have a very strong credit history, with a sc...",Credit card or prepaid card,1
6995,6995,I am a victim of identity theft and this debt ...,Debt collection,0
6996,6996,Disputed transaction for unauthorized charge b...,Credit card or prepaid card,1
6997,6997,On XX/XX/2018 I was marked for {$620.00} on an...,Debt collection,0


In [12]:
def encode_genre(genre):
    if genre == "Debt collection":
        return 1
    elif genre == "Credit card or prepaid card":
        return 2
    elif genre == "Mortgage":
        return 3
    elif genre == "Checking or savings account":
        return 4
    elif genre == "Student loan":
        return 5
    elif genre == "Vehicle loan or lease":
        return 6

df['Product1'] = df['Product'].apply(encode_genre)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['complaint'], df['Product1'],test_size=0.1, random_state=0)

print('Load %d training examples and %d validation examples. \n' %(X_train.shape[0],X_test.shape[0]))
print('Show a review in the training set : \n', X_train.iloc[10])
X_train,y_train

Load 6299 training examples and 700 validation examples. 

Show a review in the training set : 
 In XX/XX/XXXX, I received a loan modification for the mortgage on my home that I have lived in for 22 years. It was a principal and interest only loan, with my monthly payment totaling {$440.00}. The loan was made possible by extending the years on my previous mortgage, which thereby lowered my payments. The interest rate was fixed for a 30-year term. In order to be accepted for this modification, I had to go through a 3 month trial period to ensure full payments of {$440.00} were made by the first of each month. I met this requirement and therefore, my mortgage payment stayed at the rate of {$440.00} for the remainder ofthe loan. I knew that this was a principal and interest loan, and that I was responsible for paying real estate taxes and home owner 's insurance, as was directed on the signed contract. I continued to pay my taxes and home insurance and am current on all payments up to thi

(6199    Lockhart Morris an unknown company collecting ...
 72      On XX/XX/XXXX, I was notified that my new Citi...
 2571    Back in XXXX of 2019 Sychrony bank lowered my ...
 6270    Mortgage company fraudulently debited bank acc...
 3608    XX/XX/XXXX received call we let go to voicemai...
                               ...                        
 4931    My card was charged in Fall of 2018 for charge...
 3264    i been receipting numerous calls from Mid West...
 1653    XX/XX/20 account was paid in a settlement with...
 2607    I have contacted the collection agency AD Astr...
 2732    XX/XX/2020 this company has not sent any lette...
 Name: complaint, Length: 6299, dtype: object,
 6199    1
 72      2
 2571    2
 6270    3
 3608    2
        ..
 4931    2
 3264    1
 1653    1
 2607    1
 2732    1
 Name: Product1, Length: 6299, dtype: int64)

In [15]:
def cleanText(raw_text, remove_stopwords=False, stemming=False, split_text=False):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'html.parser').get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    words = letters_only.lower().split() 
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    if stemming==True:

        stemmer = SnowballStemmer('english') 
        words = [stemmer.stem(w) for w in words]
        
    if split_text==True:
        return (words)
    
    return( " ".join(words))

In [16]:

import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag
from bs4 import BeautifulSoup
import logging
from wordcloud import WordCloud
from gensim.models import word2vec
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

X_train_cleaned = []
X_test_cleaned = []

for d in X_train:
    X_train_cleaned.append(cleanText(d))
print('Show a cleaned review in the training set : \n',  X_train_cleaned[10])
    
for d in X_test:
    X_test_cleaned.append(cleanText(d))

Show a cleaned review in the training set : 
 in xx xx xxxx i received a loan modification for the mortgage on my home that i have lived in for years it was a principal and interest only loan with my monthly payment totaling the loan was made possible by extending the years on my previous mortgage which thereby lowered my payments the interest rate was fixed for a year term in order to be accepted for this modification i had to go through a month trial period to ensure full payments of were made by the first of each month i met this requirement and therefore my mortgage payment stayed at the rate of for the remainder ofthe loan i knew that this was a principal and interest loan and that i was responsible for paying real estate taxes and home owner s insurance as was directed on the signed contract i continued to pay my taxes and home insurance and am current on all payments up to this date xx xx approximately a little over a year after my xx xx xxxx modification started i received a bi

In [17]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
countVect = CountVectorizer() 
X_train_countVect = countVect.fit_transform(X_train_cleaned)
print("Number of features : %d \n" %len(countVect.get_feature_names()))
print("Show some feature names : \n", countVect.get_feature_names()[::1000])

Number of features : 15777 

Show some feature names : 
 ['aa', 'asset', 'capacity', 'contraband', 'disappointment', 'exceptionally', 'gen', 'incapable', 'let', 'mustve', 'pennys', 'randomly', 'rig', 'speedy', 'tht', 'verifiable']


In [18]:
X_train_countVect.shape,y_train.shape

((6299, 15777), (6299,))

In [19]:
mnb = MultinomialNB()
mnb.fit(X_train_countVect, y_train)

MultinomialNB()

In [20]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,roc_auc_score
def modelEvaluation(predictions):
    '''
    Print model evaluation to predicted result 
    '''
    print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(y_test, predictions)))
    print("\nClassification report : \n", metrics.classification_report(y_test, predictions))
    print("\nConfusion Matrix : \n", metrics.confusion_matrix(y_test, predictions))

In [ ]:
predictions = mnb.predict(countVect.transform(X_test_cleaned))
modelEvaluation(predictions)


Accuracy on validation set: 0.8729

Classification report : 
               precision    recall  f1-score   support

           1       0.91      0.91      0.91       274
           2       0.85      0.87      0.86       152
           3       0.85      0.99      0.91       112
           4       0.90      0.82      0.86        87
           5       0.76      0.76      0.76        37
           6       0.81      0.55      0.66        38

    accuracy                           0.87       700
   macro avg       0.85      0.82      0.83       700
weighted avg       0.87      0.87      0.87       700


Confusion Matrix : 
 [[248   6  10   5   2   3]
 [ 11 132   1   3   4   1]
 [  0   1 111   0   0   0]
 [  4  11   0  71   0   1]
 [  5   2   2   0  28   0]
 [  4   3   7   0   3  21]]


In [21]:
X_train_countVect

<6299x15777 sparse matrix of type '<class 'numpy.int64'>'
	with 631752 stored elements in Compressed Sparse Row format>

In [22]:
import pickle

In [23]:
pickle.dump(mnb,open('Naive_Bayes_model.pkl','wb'))

In [24]:
import pickle

In [25]:
pickle.dump(countVect,open('countVect_Genre.pkl','wb'))